In [31]:
import pickle
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import numpy as np
import os
import pandas as pd
from sklearn import metrics

In [3]:
def __get_data_tensor(texts, label_type, source_type):
    
    TOKENIZER_PATH = f"D:\\peter_devine_projects\\replication_package\\AIRE19_models\\tokenizer\\tokenizer_Category.{label_type}_Language.ENGLISH_Source.{source_type}_.pkl"
    MAX_SEQ_LEN = 200
    # finally, vectorize the text samples into a 2D integer tensor

    with open(TOKENIZER_PATH, 'rb') as handle:
        tokenizer = pickle.load(handle)

    sequences = tokenizer.texts_to_sequences(texts)    

    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    return pad_sequences(sequences, maxlen=MAX_SEQ_LEN), word_index

In [4]:
def get_ypreds(texts, label_type, source_type):
    
    model = load_model(f'D:\\peter_devine_projects\\replication_package\\AIRE19_models\\model\\model_Language.ENGLISH_Category.{label_type}_Source.{source_type}.h5')

    data, word_index = __get_data_tensor(texts, label_type, source_type)

    yp = model.predict(data, batch_size=32, verbose=0)
    
    return yp

In [34]:
def get_combined_ypreds(texts, label_type):
    
    yp_app = get_ypreds(texts, label_type, "APP")
    yp_twitter = get_ypreds(texts, label_type, "TWITTER")
    
    return (yp_app + yp_twitter) / 2

In [63]:
label_maps = {
    "chen_2014": [(["non-informative"], ["IRRELEVANT"])],
    "ciurumelea_2017": [(["OTHER"], ["IRRELEVANT"])],
    "di_sorbo_2016": [(["BUG"], ["PROBLEM"]), (["QUESTION", "REQUEST"], ["INQUIRY"])],
    "guzman_2015": [(["Bug report"], ["PROBLEM"]), (["User request"], ["INQUIRY"])],
    "maalej_2016": [(["Bug"], ["PROBLEM"]), (["Feature"], ["INQUIRY"])],
    "scalabrino_2017": [(["BUG"], ["PROBLEM"]), (["FEATURE"], ["INQUIRY"])],
    "tizard_2019": [(["apparent bug"], ["PROBLEM"]), (["feature request", "question on application", "help seeking", "requesting more information", "question on background"], ["INQUIRY"])],
    "williams_2017": [(["bug"], ["PROBLEM"]), (["fea"], ["INQUIRY"])],
}

In [64]:
def get_logits(texts, label_types, source_type):
    combined_logits = []
    for label_type in label_types:
        
        if source_type ==  "COMBINED":
            model_logits = get_combined_ypreds(texts, label_type)
        else:
            model_logits = get_ypreds(texts, label_type, source_type)
            
        positive_class_logits = model_logits[:, 1]
        
        combined_logits.append(positive_class_logits)
        
    return np.stack(combined_logits, axis=1)

In [91]:
raw_data_dir = "./data/raw"

supervised_cls_data_dir = "./data/supervised_cls_preds"

stanik_cls_data_dir = os.path.join(supervised_cls_data_dir, "stanik")
os.makedirs(supervised_cls_data_dir, exist_ok=True)
os.makedirs(stanik_cls_data_dir, exist_ok=True)

results = {}

# Read the files one by one, exporting them to xml
for dataset_app_name in os.listdir(raw_data_dir):
    print(dataset_app_name)
    df = pd.read_csv(os.path.join(raw_data_dir, dataset_app_name), index_col = 0)
    df.labels = df.labels.str.replace("'", "").str.replace("\]", "").str.replace("\[", "").str.split(",")
    df.labels = df.labels.apply(lambda x: [y.strip() for y in x])
    
    dataset_name = [dataset for dataset in label_maps.keys() if dataset in dataset_app_name][0]
    label_map = label_maps[dataset_name]

    true_label_set = [labels[0] for labels in label_map]
    prediction_label_set = [labels[1][0] for labels in label_map]

    true_labels = df.labels.apply(lambda x: [any([true_label in x for true_label in true_label_list]) for true_label_list in true_label_set])
    true_labels = np.asarray(true_labels.values.reshape(-1).tolist())
    
    is_all_true_cols = ~(~true_labels).all(axis=0)
    
    true_labels = true_labels[:, is_all_true_cols]
    
    app_logits = get_logits(df.text, prediction_label_set, 'APP')
    twitter_logits = get_logits(df.text, prediction_label_set, 'TWITTER')
    combined_logits = get_logits(df.text, prediction_label_set, 'COMBINED')
    
    app_logits = app_logits[:, is_all_true_cols]
    twitter_logits = twitter_logits[:, is_all_true_cols]
    combined_logits = combined_logits[:, is_all_true_cols]
    
    app_roc_auc = metrics.roc_auc_score(true_labels, app_logits)
    twitter_roc_auc = metrics.roc_auc_score(true_labels, twitter_logits)
    combined_roc_auc = metrics.roc_auc_score(true_labels, combined_logits)
    
    results[dataset_app_name[:-4]] = {
        "Stanik app model": app_roc_auc,
        "Stanik twitter model": twitter_roc_auc,
        "Stanik combined model": combined_roc_auc,
    }

chen_2014_facebook.csv
Found 8673 unique tokens.


c:\temp\python_envs\clustering_env\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  from ipykernel import kernelapp as app


Found 11298 unique tokens.
Found 8673 unique tokens.
Found 11298 unique tokens.
chen_2014_swiftkey.csv
Found 8673 unique tokens.


c:\temp\python_envs\clustering_env\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  from ipykernel import kernelapp as app


Found 11298 unique tokens.
Found 8673 unique tokens.
Found 11298 unique tokens.
chen_2014_tapfish.csv
Found 8673 unique tokens.


...

Found 6142 unique tokens.
Found 10196 unique tokens.
Found 6835 unique tokens.
Found 6943 unique tokens.
Found 10196 unique tokens.
Found 6142 unique tokens.
Found 6835 unique tokens.
williams_2017_@whatsapp.csv
Found 6943 unique tokens.

c:\temp\python_envs\clustering_env\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  from ipykernel import kernelapp as app



Found 6142 unique tokens.
Found 10196 unique tokens.
Found 6835 unique tokens.
Found 6943 unique tokens.
Found 10196 unique tokens.
Found 6142 unique tokens.
Found 6835 unique tokens.
williams_2017_@windows.csv
Found 6943 unique tokens.


c:\temp\python_envs\clustering_env\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  from ipykernel import kernelapp as app


Found 6142 unique tokens.
Found 10196 unique tokens.
Found 6835 unique tokens.
Found 6943 unique tokens.
Found 10196 unique tokens.
Found 6142 unique tokens.
Found 6835 unique tokens.


In [93]:
pd.DataFrame(results).to_csv(os.path.join(stanik_cls_data_dir, "results.csv"))

In [92]:
pd.DataFrame(results)

,chen_2014_facebook,chen_2014_swiftkey,chen_2014_tapfish,chen_2014_templerun2,ciurumelea_2017_2048,ciurumelea_2017_A Comic Viewer,ciurumelea_2017_Abstract Art,ciurumelea_2017_AcDisplay,ciurumelea_2017_Adblock Plus,ciurumelea_2017_Amaze File Manager,...,williams_2017_@android,williams_2017_@applesupport,williams_2017_@callofduty,williams_2017_@googlechrome,williams_2017_@instagram,williams_2017_@minecraft,williams_2017_@snapchat,williams_2017_@visualstudio,williams_2017_@whatsapp,williams_2017_@windows
Stanik app model,0.659085,0.737586,0.860300,0.827857,0.43750,0.498412,0.716159,0.574402,0.546154,0.330882,...,0.701544,0.639280,0.674781,0.669079,0.673102,0.729534,0.686397,0.645220,0.706343,0.657469
Stanik twitter model,0.642786,0.625225,0.702898,0.729200,0.28125,0.561078,0.731394,0.497411,0.741538,0.411765,...,0.696873,0.591651,0.621373,0.649859,0.663043,0.677039,0.658754,0.650913,0.624752,0.656430
Stanik combined model,0.664099,0.718505,0.840091,0.819743,0.37500,0.531410,0.742882,0.540205,0.621538,0.352941,...,0.720658,0.630052,0.658320,0.679731,0.692840,0.730812,0.691402,0.672096,0.691904,0.683462
